In [1]:
import bert
from bert import modeling
import numpy as np
import json
import tensorflow as tf

In [2]:
BERT_INIT_CHKPNT = 'pretraining_output/model.ckpt-2000002'

In [3]:
bert_config = modeling.BertConfig.from_json_file('BASE_config.json')

In [4]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.int32, [None, None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=True,
            input_ids=self.X,
            input_mask=self.input_masks,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        embedding = model.get_embedding_table()
        
        with tf.variable_scope('cls/predictions'):
            with tf.variable_scope('transform'):
                input_tensor = tf.layers.dense(
                    output_layer,
                    units = bert_config.hidden_size,
                    activation = modeling.get_activation(bert_config.hidden_act),
                    kernel_initializer = modeling.create_initializer(
                        bert_config.initializer_range
                    ),
                )
                input_tensor = modeling.layer_norm(input_tensor)
            
            output_bias = tf.get_variable(
            'output_bias',
            shape = [bert_config.vocab_size],
            initializer = tf.zeros_initializer(),
            )
            logits = tf.matmul(input_tensor, embedding, transpose_b = True)

In [5]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
cls = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'cls')
saver = tf.train.Saver(var_list = var_lists + cls)
saver.restore(sess, BERT_INIT_CHKPNT)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Restoring parameters from ../tiny-bert/bert-tiny-v1/model.ckpt


In [6]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'bert-base-v3/model.ckpt')

'tiny-bert-v1/model.ckpt'

In [7]:
!cp ../tiny-bert/sp10m.cased.bert* bert-base-v3
!cp ../tiny-bert/tf-config.json bert-base-v3/config.json
!tar cvzf bert-bahasa-03-04-2020.tar.gz bert-base-v3

In [11]:
import boto3

bucketName = 'huseinhouse-storage'
Key = 'bert-bahasa-03-04-2020.tar.gz'
outPutname = "bert-bahasa/bert-bahasa-03-04-2020.tar.gz"

s3 = boto3.client('s3')
s3.upload_file(Key,bucketName,outPutname)